In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
import pmdarima as pm
from pmdarima import auto_arima,arima
import warnings
# get functions from utils.py
from utils import train_data,eval_metrics,plot_train_test
from statsmodels.tsa.statespace.sarimax import SARIMAX
import joblib
from joblib import dump, load
import gc
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

In [ ]:
ari = pd.read_csv("data_ari.csv",sep=",",dtype={'location':str,'year_week':str,
                                                'value':np.float32,'relative_humidity_2m':np.float64,
                                                'temperature_2m_max':np.float64,'temperature_2m_min':np.float64},
                                                parse_dates=['truth_date'])


In [ ]:
ili = pd.read_csv("data_ili.csv",sep=",",dtype={'location':str,'year_week':str,
                                                'value':np.float32,'relative_humidity_2m':np.float64,
                                                'temperature_2m_max':np.float64,'temperature_2m_min':np.float64},
                                                parse_dates=['truth_date'])
ili = ili.drop(columns=['Unnamed: 0']).reset_index(drop=True)

In [ ]:
mape_ari = pd.DataFrame(columns=['location','model','prediction_window','mae','rmse'])
mape_ili = pd.DataFrame(columns=['location','model','prediction_window','mae','rmse'])

In [ ]:
def forecast_arima_sarima_model(train, test, mape,model,order_model,seasonal_order_model, model_name="no_model_def", country="no_country_def",exogenous_var=None):
    test_aux = test.copy()

    # Prepare prediction columns
    for h in range(4):
        test_aux[f"prediction_{h+1}_weeks"] = np.nan

    # Rolling forecast
    for i in range(len(test_aux)):
        # Combine train and observed test values so far
        train_series = pd.concat([train["value"], test_aux.iloc[:i]["value"]])
        if exogenous_var is not None:
            exog_train = pd.concat([train[exogenous_var], test_aux.iloc[:i][exogenous_var]])
            exog_forecast = test_aux.iloc[i:i+4][exogenous_var]
        else:
            exog_train = None
            exog_forecast = None        

        # Fit model
        model = SARIMAX(train_series, order=order_model, seasonal_order=seasonal_order_model,exog=exog_train)
        model_fit = model.fit(disp=False)
        
        # Forecast 1 to 4 weeks ahead, or less at the end
        forecast_steps = min(4, len(test_aux) - i)
        if exogenous_var is not None:
            forecast = model_fit.forecast(steps=forecast_steps, exog=exog_forecast.iloc[:forecast_steps])
        else:
            forecast = model_fit.forecast(steps=forecast_steps)
        # Save forecasted values
        #for h, pred in enumerate(forecast):
        #    test_aux.loc[test_aux.index[i + h], f"prediction_{h+1}_weeks"] = pred
        
        for h in range(forecast_steps):
            test_aux.loc[test_aux.index[i + h], f"prediction_{h+1}_weeks"] = forecast.iloc[h]

    # Evaluate predictions
    for h in range(4):
        shifted = test_aux["value"].shift(-h)
        preds = test_aux[f"prediction_{h+1}_weeks"]
        valid_idx = ~shifted.isna()
        y_true = shifted[valid_idx]
        y_pred = preds[valid_idx]
        resid = y_true - y_pred
        residual = y_pred - y_true
        test_aux.loc[valid_idx, f"week_{h+1}_res"] = resid
        mae, rmse = eval_metrics(y_true, y_pred)
        mape = pd.concat([
            mape,
            pd.DataFrame([[country, model_name, f"{h+1}_week", mae, rmse]],
                         columns=['location', 'model', 'prediction_window', 'mae', 'rmse'])
        ], ignore_index=True)
        print(mape.tail(4))
    return mape, test_aux,residual


In [ ]:
name_ili = ili['location'].unique()
name_ari = ari['location'].unique()

In [ ]:

loaded_model = joblib.load(f'models/arima_model_RO_ILI.joblib')
order_model = loaded_model.order
seasonal_order_model = loaded_model.seasonal_order

In [ ]:
def create_features(data):
    """
    Create additional features for the non sequential dataset.
    """
    data = data.copy()

    # Extract year, month, day, weekday, and week from 'truth_date'
    data['year'] = data.index.year
    data['month'] = data.index.month

    week = data['year_week'].str.split('-W').str[1]
    data['week'] = week.astype(int)
    for h in range(1,5):
        data[f'lag_value_{h}'] = data['week_1_res'].shift(h)
        data[f'lag_humidity_{h}'] = data['relative_humidity_2m'].shift(h)
        data[f'lag_temp_max_{h}'] = data['temperature_2m_max'].shift(h)
        data[f'lag_temp_min_{h}'] = data['temperature_2m_min'].shift(h)
    data = data.dropna()
    # Convert cyclical categorical variables to category type
    data['month_sin'] = np.sin(2 * np.pi * data['month']/12)
    data['month_cos'] = np.cos(2 * np.pi * data['month']/12)
    data['week_sin'] = np.sin(2 * np.pi * data['week']/52)
    data['week_cos'] = np.cos(2 * np.pi * data['week']/52)
    data = data.drop(columns=['month', 'week','year_week'])
    return data

In [ ]:
def plot_train_test(train, test, model_name, location,folder):
    plt.figure(figsize=(15, 7))

    # Plot actual data
    plt.plot(train.index, train["value"], color='blue', label='Training Data')
    plt.plot(test.index, test["value"], color='orange', label='Actual Test Data')

    # Unified forecast column names
    forecast_horizons = {
        "prediction_1_weeks": ("1 Week Ahead", "green", 0),
        "prediction_2_weeks": ("2 Weeks Ahead", "red", 1),
        "prediction_3_weeks": ("3 Weeks Ahead", "purple", 2),
        "prediction_4_weeks": ("4 Weeks Ahead", "brown", 3),
    }

    for col, (label, color, shift_val) in forecast_horizons.items():
        if col in test.columns:
            # Shift predictions forward by their horizon
            plt.plot(test.index, test[col].shift(shift_val), linestyle='--', color=color, label=f'{model_name} {label}')

    plt.legend(loc='upper left')
    plt.title(f"{model_name} Forecasting – {location}")
    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.grid(True)
    plt.tight_layout()

    # Save figure
    plt.savefig(f"plots_{folder}/plot_{location}_{model_name}.jpg", format='jpg', dpi=300)

In [ ]:
def rf_variable_selection_and_hyperparam_tuning(train,test,country =None, model_name=None, mape=None):
    X = train.drop(columns=['week_1_res','week_2_res','week_3_res','week_4_res'])
    y = train[['week_1_res','week_2_res','week_3_res','week_4_res']]

    X_test = test.drop(columns=['week_1_res','week_2_res','week_3_res','week_4_res'])
    y_test= test[['week_1_res','week_2_res','week_3_res','week_4_res']]
    

    base_rf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=2332))
    base_rf.fit(X, y)

    importances = np.array([est.feature_importances_ for est in base_rf.estimators_])

    mean_importances = importances.mean(axis=0)


    selected_mask = mean_importances >= 0.01
    selected_features = X.columns[selected_mask].tolist()
    X_selected = X[selected_features]

    # Step 3: Hyperparameter tuning with RandomizedSearchCV
    param_grid = {
        'n_estimators': [100, 200, 300, 400, 500],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    }

    tscv = TimeSeriesSplit(n_splits=5)
    rf = RandomForestRegressor(random_state=2332)

    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_grid,
        n_iter=20,
        cv=tscv,
        n_jobs=-1,
        scoring='neg_mean_absolute_error',
        random_state=2332
    )

    random_search.fit(X_selected, y)

    best_model = random_search.best_estimator_
    model_final = MultiOutputRegressor(best_model)
    model_final.fit(X_selected, y)
    dump(model_final, f"models_rf/rf_arima_sarima_model_{country}_{model_name}.joblib")
    test_aux = test.copy()
    prediction_columns = [f"prediction_{h+1}_weeks" for h in range(4)]
    preds = model_final.predict(X_test[selected_features])
    test_aux[prediction_columns] = preds

    # Evaluate predictions
    test_aux = test_aux.dropna()
    mae0, rmse0 = eval_metrics(test_aux["week_1_res"], test_aux["prediction_1_weeks"])
    mae1, rmse1  = eval_metrics(test_aux["week_2_res"], test_aux["prediction_2_weeks"])
    mae2, rmse2  = eval_metrics(test_aux["week_3_res"], test_aux["prediction_3_weeks"])
    mae3, rmse3 = eval_metrics(test_aux["week_4_res"], test_aux["prediction_4_weeks"])

    mape = pd.concat([
    mape,
    pd.DataFrame([
        [country, model_name, "1_week", mae0, rmse0],
        [country, model_name, "2_week", mae1, rmse1],
        [country, model_name, "3_week", mae2, rmse2],
        [country, model_name, "4_week", mae3, rmse3]
    ], columns=['location', 'model', 'prediction_window', 'mae', 'rmse'])
], ignore_index=True)
    return model_final, selected_features, test_aux,mape


In [ ]:
name_ari = ari['location'].unique()

In [ ]:
name_ili = ili['location'].unique()

In [ ]:
name_ili
#problem DK PL AT HR GR IE LU LT NL NO MT

['SI', 'EE', 'FR', 'RO', 'HU','LV', 'BE']
['LU', 'LT', 'NL', 'CZ', 'NO', 'MT']

In [ ]:
mape_ili2 = pd.DataFrame(columns=['location','model','prediction_window','mae','rmse'])

In [ ]:
mape_ili2

In [ ]:
for i in name_ili:
    print(f'the country running is {i}')
    loaded_model = joblib.load(f'models/sarima_model_{i}_ILI.joblib')
    order_model = loaded_model.order
    seasonal_order_model = loaded_model.seasonal_order
    data = ili[ili['location']==i].copy()
    train, test = train_data(ili,i, "2023-10-15")
    mape_train, train2,res_1 = forecast_arima_sarima_model(train, train, mape_ili, loaded_model,order_model=order_model,seasonal_order_model=seasonal_order_model, model_name="ARIMA", country=i)
    mape_arima, test_predictions,res_2 = forecast_arima_sarima_model(train, test, mape_ili, loaded_model,order_model=order_model,seasonal_order_model=seasonal_order_model, model_name="ARIMA", country=i)
    train3= train2[[ 'year_week', 'relative_humidity_2m',
       'temperature_2m_max', 'temperature_2m_min', 'covid',
       'week_1_res', 'week_2_res', 'week_3_res',
       'week_4_res']]
    test_predictions_2 = test_predictions[[ 'year_week', 'relative_humidity_2m',
       'temperature_2m_max', 'temperature_2m_min', 'covid',
       'week_1_res', 'week_2_res', 'week_3_res',
       'week_4_res']]
    train_use = create_features(train3)
    test_use = create_features(test_predictions_2)
    print(f'process random forest for {i}')
    model_final, selected_features, test_predictions_rf,mape = rf_variable_selection_and_hyperparam_tuning(train_use,test_use,country=i, model_name="ARIMA")
    test_predictions_values = test_predictions[['value', 'prediction_1_weeks', 'prediction_2_weeks',
       'prediction_3_weeks', 'prediction_4_weeks']].copy()
    res_prediction = test_predictions_rf[['prediction_1_weeks', 'prediction_2_weeks',
            'prediction_3_weeks', 'prediction_4_weeks']].copy()
    combined_predictions = test_predictions_values[['prediction_1_weeks', 'prediction_2_weeks',
                                                      'prediction_3_weeks', 'prediction_4_weeks']] + \
    res_prediction[['prediction_1_weeks', 'prediction_2_weeks',
                                             'prediction_3_weeks', 'prediction_4_weeks']]
    combined_df = test_predictions_values[['value']].copy()
    combined_df[["prediction_1_weeks", "prediction_2_weeks",
                  "prediction_3_weeks", "prediction_4_weeks"]] = combined_predictions
    plot_train_test(train,combined_df,'mixed_test_ILI',i,'RF')
    combined_df.to_csv(f'mixed_model_{i}_ILI.csv')
   # Evaluate predictions
    for h in range(4):
       shifted = combined_df["value"].shift(-h)
       preds = combined_df[f"prediction_{h+1}_weeks"]
       valid_idx = ~shifted.isna()
       y_true = shifted[valid_idx]
       y_pred = preds[valid_idx]
       mae, rmse = eval_metrics(y_true, y_pred)
       mape_ili2 = pd.concat([
          mape_ili2,
          pd.DataFrame([[i, 'rf/arima', f"{h+1}_week", mae, rmse]],
                           columns=['location', 'model', 'prediction_window', 'mae', 'rmse'])
       ], ignore_index=True)

In [ ]:
mape_ili2

In [ ]:
mape_ili2.to_csv('mape_mixed_ili2.csv',sep=';',decimal=',')

In [ ]:
mape_ili2 = pd.read_csv('mape_mixed_ili2.csv',sep=';',decimal=',')

In [ ]:
mape_ili2

In [ ]:
def xgboost_variable_selection_and_hyperparam_tuning(train,test,country =None, model_name=None, mape=None):
    X = train.drop(columns=['week_1_res','week_2_res','week_3_res','week_4_res'])
    y = train[['week_1_res','week_2_res','week_3_res','week_4_res']]

    X_test = test.drop(columns=['week_1_res','week_2_res','week_3_res','week_4_res'])
    y_test= test[['week_1_res','week_2_res','week_3_res','week_4_res']]
    

    base_model =MultiOutputRegressor(XGBRegressor(objective='reg:squarederror', random_state=2332, n_jobs=-1))

    base_model.fit(X, y)

    importances = np.array([est.feature_importances_ for est in base_model.estimators_])

    mean_importances = importances.mean(axis=0)


    selected_mask = mean_importances >= 0.01
    selected_features = X.columns[selected_mask].tolist()
    X_selected = X[selected_features]

    # Step 3: Hyperparameter tuning with RandomizedSearchCV
    param_grid = {
        "estimator__n_estimators": [50, 100, 200],
        "estimator__max_depth": [3, 4, 5, 10],
        "estimator__learning_rate": np.linspace(0.01, 0.1, 10),
        "estimator__subsample": [0.8, 1.0],
        "estimator__colsample_bytree": [0.7, 0.8, 1.0],       # feature subsampling
        "estimator__min_child_weight": [1, 3, 5],              # min data needed in a child
        "estimator__gamma": [0, 0.1, 0.3, 0.5],                # min loss reduction to split
        "estimator__reg_alpha": np.linspace(0, 1, 5),          # L1 regularization
        "estimator__reg_lambda": np.linspace(0, 1, 5),         # L2 regularization
    }

    tscv = TimeSeriesSplit(n_splits=5)

    random_search = RandomizedSearchCV(
        estimator=base_model,
        param_distributions=param_grid,
        n_iter=20,
        cv=tscv,
        n_jobs=-1,
        scoring='neg_mean_absolute_error',
        random_state=2332
    )

    random_search.fit(X_selected, y)

    model_final = random_search.best_estimator_
    model_final.fit(X_selected, y)
    dump(model_final, f"models_xgboost/xgboost_mixed_model_{country}_{model_name}.joblib")
    test_aux = test.copy()
    prediction_columns = [f"prediction_{h+1}_weeks" for h in range(4)]
    preds = model_final.predict(X_test[selected_features])
    test_aux[prediction_columns] = preds

    # Evaluate predictions
    test_aux = test_aux.dropna()
    mae0, rmse0 = eval_metrics(test_aux["week_1_res"], test_aux["prediction_1_weeks"])
    mae1, rmse1  = eval_metrics(test_aux["week_2_res"], test_aux["prediction_2_weeks"])
    mae2, rmse2  = eval_metrics(test_aux["week_3_res"], test_aux["prediction_3_weeks"])
    mae3, rmse3 = eval_metrics(test_aux["week_4_res"], test_aux["prediction_4_weeks"])

    mape = pd.concat([
    mape,
    pd.DataFrame([
        [country, model_name, "1_week", mae0, rmse0],
        [country, model_name, "2_week", mae1, rmse1],
        [country, model_name, "3_week", mae2, rmse2],
        [country, model_name, "4_week", mae3, rmse3]
    ], columns=['location', 'model', 'prediction_window', 'mae', 'rmse'])
], ignore_index=True)
    return model_final, selected_features, test_aux,mape


In [ ]:
for i in name_ari:
    print(f'the country running is {i}')
    loaded_model = joblib.load(f'models/sarima_model_{i}_ARI.joblib')
    order_model = loaded_model.order
    seasonal_order_model = loaded_model.seasonal_order
    data = ari[ari['location']==i].copy()
    train, test = train_data(ari,i, "2023-10-15")
    mape_train, train2,res_1 = forecast_arima_sarima_model(train, train, mape_ili, loaded_model,order_model=order_model,seasonal_order_model=seasonal_order_model, model_name="ARIMA", country=i)
    mape_arima, test_predictions,res_2 = forecast_arima_sarima_model(train, test, mape_ili, loaded_model,order_model=order_model,seasonal_order_model=seasonal_order_model, model_name="ARIMA", country=i)
    train3= train2[[ 'year_week', 'relative_humidity_2m',
       'temperature_2m_max', 'temperature_2m_min', 'covid',
       'week_1_res', 'week_2_res', 'week_3_res',
       'week_4_res']]
    test_predictions_2 = test_predictions[[ 'year_week', 'relative_humidity_2m',
       'temperature_2m_max', 'temperature_2m_min', 'covid',
       'week_1_res', 'week_2_res', 'week_3_res',
       'week_4_res']]
    train_use = create_features(train3)
    test_use = create_features(test_predictions_2)
    print(f'process random forest for {i}')
    model_final, selected_features, test_predictions_rf,mape = xgboost_variable_selection_and_hyperparam_tuning(train_use,test_use,country=i, model_name="ARIMA")
    test_predictions_values = test_predictions[['value', 'prediction_1_weeks', 'prediction_2_weeks',
       'prediction_3_weeks', 'prediction_4_weeks']].copy()
    res_prediction = test_predictions_rf[['prediction_1_weeks', 'prediction_2_weeks',
            'prediction_3_weeks', 'prediction_4_weeks']].copy()
    combined_predictions = test_predictions_values[['prediction_1_weeks', 'prediction_2_weeks',
                                                      'prediction_3_weeks', 'prediction_4_weeks']] + \
    res_prediction[['prediction_1_weeks', 'prediction_2_weeks',
                                             'prediction_3_weeks', 'prediction_4_weeks']]
    combined_df = test_predictions_values[['value']].copy()
    combined_df[["prediction_1_weeks", "prediction_2_weeks",
                  "prediction_3_weeks", "prediction_4_weeks"]] = combined_predictions
    plot_train_test(train,combined_df,'mixed_xgboost_test_ARI',i,'RF')
    combined_df.to_csv(f'mixed_xgboost_model_{i}_ARI.csv')
   # Evaluate predictions
    for h in range(4):
       shifted = combined_df["value"].shift(-h)
       preds = combined_df[f"prediction_{h+1}_weeks"]
       valid_idx = ~shifted.isna()
       y_true = shifted[valid_idx]
       y_pred = preds[valid_idx]
       mae, rmse = eval_metrics(y_true, y_pred)
       mape_ili2 = pd.concat([
          mape_ili2,
          pd.DataFrame([[i, 'rf/arima', f"{h+1}_week", mae, rmse]],
                           columns=['location', 'model', 'prediction_window', 'mae', 'rmse'])
       ], ignore_index=True)

In [ ]:
mape_ili2

In [ ]:
mape_ili2.to_csv('mape_mixed_xgboost_ari2.csv',sep=';',decimal=',')